# Voltametria de corrente amostrada
A voltametria de corrente amostrada consiste de uma série de saltos de potenciais nos quais a corrente é amostrada em um tempo pré-determinado $\tau_s$. Portanto, as equações a serem resolvidas são as mesmas do exemplo anterior. A diferença é que a solução é obtida para uma sequência de potenciais entre $E_i$ e $E_f$ e a corrente é representada em função de $E$.

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import math
from scipy import special

## Definindo o modelo

Definindo a variável do modelo e a qual domínio ela pertence (neste caso é necessário calcular a concentração de b para aplicar a equação de Nernst na interface):

In [2]:
model = pybamm.BaseModel()

co = pybamm.Variable("Concentration of O", domain="electrolyte") 
cr = pybamm.Variable("Concentration of R", domain="electrolyte")

Definindo os parâmetros do modelo:

In [3]:
eta = pybamm.Parameter("Applied Overpotential [V]") # sobrepotencial aplicado
F = pybamm.Parameter("Faraday constant [C.mol-1]")
R = pybamm.Parameter("Molar gas constant [J.K-1.mol-1]")
T = pybamm.Parameter("Temperature [K]")

As equações são definidas em termos do fluxo e do divergente e adicionadas ao dicionário `model.rhs`

In [4]:
No = - pybamm.grad(co)  # define o fluxo de O
Nr = -pybamm.grad(cr) # define o fluxo de R
rhso = -pybamm.div(No)  # define o lado direito da equação 1.0a
rhsr = -pybamm.div(Nr) # define o lado direito da equação 1.0b

model.rhs = {co: rhso, cr: rhsr}  # adicona as equações ao dicionário

Introduzindo as condições iniciais e de contorno:

In [5]:
# condições iniciais
model.initial_conditions = {co: pybamm.Scalar(1), cr:pybamm.Scalar(0)}

# boundary conditions
f = F/(R*T)
teta = pybamm.exp(f*eta)
left_co = teta/(1+teta) # equação 1.3a. Valor de co na interface
left_cr = 1/(1+teta) # equação 1.3b. Valor de cr na interface
right_co = pybamm.Scalar(1) # equação 1.2a. Valor de co no seio da solução.
right_cr = pybamm.Scalar(0) # equação 1.2b. Valor de cr no seio da solução.
model.boundary_conditions = {co: {"left": (left_co, "Dirichlet"), "right": (right_co, "Dirichlet")},
                             cr: {"left": (left_cr, "Dirichlet"), "right": (right_cr, "Dirichlet")}}
#Dirichlet refere-se à condições de contorno que determinam o valor das variáveis co e cr na fronteira.

Adicionando as variáveis de interesse ao dicionário `model.variables`. 

In [6]:
model.variables = {"Concentration of O": co, "Flux of O": No, "Concentration of R": cr, "Flux of R": Nr}

## Usando o modelo

### Definindo a geometria e a malha

As variáveis espaciais são definidas independentemente das variáveis do modelo. O domínio 1D varia no intervalo $0 \le x \le 6$. "eta" é deixado como input para permitir simulações com diferentes sobrepotenciais aplicados.

In [7]:
param = pybamm.ParameterValues(
    {
        "Applied Overpotential [V]": "[input]",
        "Faraday constant [C.mol-1]": 96485.3,
        "Molar gas constant [J.K-1.mol-1]": 8.31446,
        "Temperature [K]": 298.15
    }
)

In [8]:
# define geometry
x = pybamm.SpatialVariable(
    "x", domain=["electrolyte"], coord_sys="cartesian"
)
geometry = {"electrolyte": {x: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(6)}}}

In [9]:
param.process_model(model)
param.process_geometry(geometry)

Criando uma malha uniforme. A implementar : malha com expansão exponencial. (ver descrição abaixo)

In [10]:
# mesh and discretise
submesh_types = {"electrolyte": pybamm.Uniform1DSubMesh}
var_pts = {x: 400}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

Example of meshes that do require parameters include the `pybamm.Exponential1DSubMesh` which clusters points close to one or both boundaries using an exponential rule. It takes a parameter which sets how closely the points are clustered together, and also lets the users select the side on which more points should be clustered. For example, to create a mesh with more nodes clustered to the right (i.e. the surface in the particle problem), using a stretch factor of 2, we pass an instance of the exponential submesh class and a dictionary of parameters into the `MeshGenerator` class as follows: `pybamm.MeshGenerator(pybamm.Exponential1DSubMesh, submesh_params={"side": "right", "stretch": 2})`

Discretizando por Volumes Finito. A testar: Elementos finitos.

In [11]:
spatial_methods = {"electrolyte": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

### Resolvendo o modelo
Você pode obter soluções para valores inseridos para o sobrepotencial

O Solver ScipySolver é escolhido. Outras opções?. Definindo a malha uniforme no tempo. Como é feita a discretização no tempo? Como tratar stiff problems?  A implementar: comparação com soluções analiticas para corrente e concentração. 

In [12]:
from ipywidgets import interact, Text, Layout, Button, Output
from IPython.display import display, clear_output

# solução
solver = pybamm.ScipySolver()
t = np.linspace(0.00001, 1, 1000)
f = 38.9217

output = Output() # define um widget output para capturar o gráfico e depois mostrá-lo com display

def plot_solution(ei,ef,e0,npts):
    with output:
        clear_output(wait=True) # limpa gráficos anteriormente gerados
        try:
            ei = float(ei)  # Garante que a entrada seja um número válido.
            ef = float(ef)
            e0 = float(e0)
            npts=int(npts)
        except ValueError:
            print("Por favor, insira um número válido")
            return
    
        interval = ef-ei
        delta = interval/npts
        es = np.arange(ei,ef+delta,delta) #potenciais dos saltos
        i_smp = [] # armazena as correntes amostradas
        for e_ap in es:
            solution = solver.solve(model, t, inputs={"Applied Overpotential [V]": e_ap-e0}) #resolve para cada potencial
            No_sol = solution["Flux of O"]
            i_smp.append(-No_sol(1,x=0)*np.sqrt(np.pi)) #divide pela corrente limite (condição de Cottrell) e armazena
        
        # plot
        fig, ax1 = plt.subplots()

        ax1.plot(es,i_smp,"r-",linewidth=1.5)
        ax1.set_xlabel("E / V")
        ax1.set_ylabel("Corrente")
        ax1.set_xlim([ef+interval/10,ei-interval/10])
        plt.tight_layout()
        plt.show()

# Cria um Text widget para número de pontos
npts_input = Text(value='5', description= 'entre no de ptos :', continuous_update=False,
                style={'description_width':'initial'})

# Cria um Text widget para entrada do potencial inicial
ei_input = Text(value='-0.5', description= 'entre $ E_i $ em V :', continuous_update=False,
                style={'description_width':'initial'})

# Cria um Text widget para entrada do potencial final
ef_input = Text(value='0.5', description= 'entre $ E_f $ em V :', continuous_update=False,
                style={'description_width':'initial'})

# Cria um Text widget para entrada do potencial padrão
e0_input = Text(value='0', description= 'entre $ E^0 $ em V :', continuous_update=False,
                style={'description_width':'initial'})

# Cria um botão para recalcular a função
run_button = Button(description="Recalcular")

# Define um manipulador de eventos para o click no botão
def on_button_click(b):
    ei = ei_input.value
    ef = ef_input.value
    e0 = e0_input.value
    npts = npts_input.value
    if ei and ef and e0 and npts:  # Ensure both values are entered
        plot_solution(ei,ef,e0,npts)
        
# Link o click do botão com o manipulador de eventos
run_button.on_click(on_button_click)

# Mostra o gráfico default na primeira execução
plot_solution(ei_input.value, ef_input.value,e0_input.value,npts_input.value)

# mostra os widgets, incluindo o output
display(ei_input, ef_input, e0_input, npts_input, run_button, output)



Text(value='-0.5', continuous_update=False, description='entre $ E_i $ em V :', style=TextStyle(description_wi…

Text(value='0.5', continuous_update=False, description='entre $ E_f $ em V :', style=TextStyle(description_wid…

Text(value='0', continuous_update=False, description='entre $ E^0 $ em V :', style=TextStyle(description_width…

Text(value='5', continuous_update=False, description='entre no de ptos :', style=TextStyle(description_width='…

Button(description='Recalcular', style=ButtonStyle())

Output()